In [2]:
import bucket_manager.bucket_manager as bm
import numpy as np

In [2]:
import os
import json
s3_host = 'https://s3.echo.stfc.ac.uk/auth/1.0'
api = 'swift'
# keys = bm.get_keys(api)
with open(os.path.expanduser('~/keys/lsst-swift-credentials.json'), 'r') as kf:
    keys = json.load(kf)
user = keys['user']
secret_key = keys['secret_key']


In [3]:
swift = bm.get_conn_swift(user, secret_key, s3_host)
swift

In [4]:
swift_service = bm.get_service_swift(user, secret_key, s3_host)
swift_service

In [5]:
bucket_list = bm.bucket_list_swift(swift)

In [6]:
bucket_list

['DRP',
 'LSST-IR-FUSION',
 'LSST-IR-FUSION-Butlers',
 'LSST-IR-FUSION-Butlers-del',
 'LSST-IR-FUSION-TESTSTRATEGY',
 'LSST-IR-FUSION-rdsip005',
 'LSST-IR-FUSION-test',
 'LSST-IR-FUSION-testtree',
 'LSST-IR-FUSION-testtreec',
 'LSST-IR-FUSION-testtreecs',
 'LSST-IR-FUSION_gen3_conversion',
 'dmu4',
 'lsst-dac',
 'lsst-drp-config',
 'lsst-test',
 'lsst-test-3',
 'lsst-test2',
 'test-large-file']

In [7]:
bucket_name = 'test-large-file'

In [8]:
large_file_path = os.path.expanduser('~/test/large_files/large-file')

In [23]:
try:
    current_objects = bm.object_list_swift(swift, bucket_name, full_listing=False)
except Exception as e:
    print(f'Error: {e}')

In [10]:
try:
    swift.put_container(bucket_name)
except Exception as e:
    print(f'Error: {e}')

In [24]:
current_objects

['/path/to/a_string',
 'rds/project/rds-rPTGgs6He74/davem/dummy_data/large_file/large-file',
 'rds/project/rds-rPTGgs6He74/davem/dummy_data/large_file/large-file/slo/1731664196.000000/6442450944/2147483648/00000000',
 'rds/project/rds-rPTGgs6He74/davem/dummy_data/large_file/large-file/slo/1731664196.000000/6442450944/2147483648/00000001',
 'rds/project/rds-rPTGgs6He74/davem/dummy_data/large_file/large-file/slo/1731664196.000000/6442450944/2147483648/00000002']

In [25]:
len(current_objects)

5

In [11]:
a_string = 'a_string'
metadata = 'somethign about a_string'

In [13]:
swift.put_object(container=bucket_name, obj='/path/to/a_string', contents=a_string, headers={'x-object-meta-zip-contents':metadata})

'7a0d3f5c88466513b32cee16e0620162'

In [18]:
bob = swift.get_object(container=bucket_name, obj='/path/to/a_string')

In [22]:
bob[0]['x-object-meta-zip-contents']

'somethign about a_string'

In [13]:
for key in current_objects:
    if type(key) == str:
        existing_zip_contents = None
        if key.endswith('.zip'):
            print('.', end='', flush=True)
            try:
                existing_zip_contents = str(swift.get_object(bucket_name,''.join([key,'.metadata']))[1].decode('UTF-8')).split('|') # use | as separator
            except Exception as e:
                try:
                    existing_zip_contents = swift.head_object(bucket_name,key)['x-object-meta-zip-contents'].split('|') # use | as separator
                    if len(existing_zip_contents) == 1:
                        existing_zip_contents = existing_zip_contents[0].split(',') # revert to comma if no | found
                except KeyError as e:
                    print('KeyError: ', e)
                except Exception as e:
                    print('Exception: ', e)
            if existing_zip_contents:
                print(existing_zip_contents)
        else:
            continue
    else:
        continue

In [ ]:
# try:
#     bm.download_file_swift(swift, bucket_name, 'butler_full_20221201/csv/viking/HSC_G/9074/collated_0.zip', os.path.expanduser('~/csv/viking/HSC_G/9074/collated_0.zip'))
# except Exception as e:
#     print(f'Error: {e}')

: 

```python
#swift api note
container = 'new-container'
    with open('local.txt', 'r') as local:
        conn.put_object(
            container,
            'local_object.txt',
            contents=local,
            content_type='text/plain'
        )
```

The following fails as put_object is low level and does not segment large files.
```python
with open(large_file_path, 'rb') as lf:
    swift.put_object(bucket_name, 'large-file', contents=lf, content_type='bytes')
```

The swiftclient.service.SwiftService object provides a higher-level interface that can do file segmentation.

In [ ]:
segment_size = 2*1024**3 # 2GiB

with open(large_file_path, 'rb') as lf:
    lf_bytes = lf.read()
    file_size = len(lf.read())
    n_segments = int(np.ceil(file_size / segment_size))
    print(n_segments)
    print(file_size)

segments = [lf_bytes for i in range(n_segments)]
    start = i * segment_size
    end = min(start + segment_size, file_size)
    print(start, end)
    segment_number = i + 1
    print(segment_number)
    with open(large_file_path, 'rb') as lf:
        segments.append(lf.read()[start:end])
        # segment_objects = [ swift_service.SwiftUploadObject(bucket_name, f'{large_file_path}_segmented_{segment_number}', contents=segment, content_type='bytes') for segment_number, segment in enumerate(segments) ]

    # swift_service.upload(bucket_name, segment_objects, options={
    #             'meta': [],
    #             'header': [],
    #             'segment_size': segment_size,
    #             'use_slo': True,
    #             'segment_container': bucket_name,
    #             'leave_segments': False,
    #             'changed': None,
    #             'skip_identical': False,
    #             'skip_container_put': False,
    #             'fail_fast': True,
    #             'dir_marker': False  # Only for None sources
    # })
    #put_object(bucket_name, 'large-file', contents=lf, content_type='bytes')

3
6442450944
0 2147483648
1
